In [5]:
# %pip install numpy
# %pip install mcts-simple

In [2]:
from our_packit import triangular_mode, hexagonal_mode

board = triangular_mode.game_logic.get_board(4)
triangular_mode.display.print_board(board) 

        /0\
      /0\0/0\
    /0\0/0\0/0\
  /0\0/0\0/0\0/0\


In [3]:
from typing import Any
from mcts_simple import Game
import numpy as np

from our_packit.triangular_mode.display import print_board
import our_packit.triangular_mode.game_logic as tgc 

class TriangularPackit(Game):
    def __init__(self, board_size):
        self.board = tgc.get_board(board_size)
        self.players = [0, 1]
        self.turn = 1
        self._update_actions()

    def _update_actions(self):
        self.possible_actions_list = tgc.get_possible_moves(self.board, self.turn)

    def render(self):
        print_board(self.board)

    def get_state(self):
        return np.copy(self.board)

    def number_of_players(self):
        return len(self.players)

    def current_player(self):
        return (self.turn + 1) % 2  

    def possible_actions(self):
        return list(range(len(self.possible_actions_list)))

    def take_action(self, action: int):
        self.board = tgc.place_polygon(self.board, self.possible_actions_list[action])
        self.turn += 1
        self._update_actions()

    def has_outcome(self):
        return len(self.possible_actions_list) == 0

    def winner(self):
        return [self.players[self.turn % 2]]
    


In [4]:
import our_packit.hexagonal_mode.game_logic as hgc 

class HexagonalPackit(Game):
    def __init__(self, board_size):
        self.board = hgc.generate_board(board_size)
        self.players = [0, 1]
        self.turn = 1
        self._update_actions()

    def _update_actions(self):
        self.possible_actions_list = hgc.get_possible_placements_for_turn(self.board, self.turn)

    def render(self):
        hgc.print_board(self.board)

    def get_state(self):
        return np.copy(self.board)

    def number_of_players(self):
        return len(self.players)

    def current_player(self):
        return (self.turn + 1) % 2  

    def possible_actions(self):
        return list(range(len(self.possible_actions_list)))

    def take_action(self, action: int):
        self.board = hgc.place_polygon(self.board, self.possible_actions_list[action])
        self.turn += 1
        self._update_actions()

    def has_outcome(self):
        return len(self.possible_actions_list) == 0

    def winner(self):
        return [self.players[self.turn % 2]]
    


In [6]:
from mcts_simple import UCT, MCTS

game = HexagonalPackit(3)

tree = UCT(game, allow_transpositions=False, training=True)
tree.self_play(iterations=10)
tree.save('HexagonalPackit301.mcts')
tree = UCT(game, allow_transpositions=False, training=False)
tree.load('HexagonalPackit301.mcts')
# tree.save("triangularPackit601.mcts")
tree.self_play()

Evaluating: 100%|██████████| 1/1 [00:00<00:00, 24.19it/s]

[0. 0. 0. 1. 1.]
[0. 0. 0. 0. 1.]
[0. 0. 0. 0. 0.]
[1. 0. 0. 0. 0.]
[1. 1. 0. 0. 0.]
[0. 0. 0. 1. 1.]
[1. 0. 0. 0. 1.]
[0. 0. 0. 0. 0.]
[1. 0. 0. 0. 0.]
[1. 1. 0. 0. 0.]
[0. 1. 0. 1. 1.]
[1. 1. 0. 0. 1.]
[0. 0. 0. 0. 0.]
[1. 0. 0. 0. 0.]
[1. 1. 0. 0. 0.]
[0. 1. 0. 1. 1.]
[1. 1. 0. 0. 1.]
[0. 0. 0. 1. 1.]
[1. 0. 0. 1. 1.]
[1. 1. 0. 0. 0.]
[0. 1. 0. 1. 1.]
[1. 1. 0. 0. 1.]
[1. 1. 0. 1. 1.]
[1. 1. 1. 1. 1.]
[1. 1. 0. 0. 0.]


In [ ]:
from mcts_simple import UCT, MCTS

game = TriangularPackit(6)

tree = UCT(game, allow_transpositions=False, training=True, c=2)
tree.self_play(iterations=10000)
tree.save("triangularPackit603.mcts")

game = TriangularPackit(6)

tree = UCT(game, allow_transpositions=False, training=True)
tree.self_play(iterations=10000)
tree.save("triangularPackit604.mcts")

Training:   0%|          | 0/10000 [00:00<?, ?it/s]

Training:  79%|███████▉  | 7907/10000 [2:21:12<41:04,  1.18s/it]   

In [73]:
import random
from copy import deepcopy
from tqdm import tqdm

wins = {
    0: 0,
    1: 0
}
game = TriangularPackit(6)
tree = MCTS(game, allow_transpositions = False, training = False)
tree.load("triangularPackit602.mcts")
num_games = 100
for i in tqdm(range(num_games)):
    human_player = i % 2
    game = TriangularPackit(6)
    # tree = MCTS(game, allow_transpositions = False, training = False)
    # tree.load("triangularPackit401.mcts")
    node = tree.root
    while not game.has_outcome():
        # game.render()
        actions = game.possible_actions()
        if game.current_player() == human_player:
            # print("Possible actions:")
            # for i, a in zip(actions, game.possible_actions_list):
            #     print(f'{i}: {a}')
            # action = int(input("> "))
            action = random.choice(actions)
            assert action in actions
            if node is not None:
                node = node.children[action] if action in node.children else None
        else:
            if node is not None and len(node.children) > 0:
                action = node.choose_best_action(tree.training)
                node = node.children[action]
            else:
                action = random.choice(actions)
                node = None
        game.take_action(action)
    # game.render()
    if 1-human_player in game.winner():
        wins[1-human_player] += 1
    
print(f'zaczynając: {wins[0] / (num_games/2)}, nie zaczynając: {wins[1] / (num_games/2)}')

100%|██████████| 100/100 [00:04<00:00, 23.73it/s]

zaczynając: 0.36, nie zaczynając: 0.74


In [5]:
import random
from copy import deepcopy
from tqdm import tqdm
from our_packit.mcts_games import HexagonalPackit
from mcts_simple import MCTS

wins = {
    0: 0,
    1: 0
}
game = HexagonalPackit(4)
tree = MCTS(game, allow_transpositions = False, training = False)
tree.load("hex41.5.mcts")
num_games = 100
for i in tqdm(range(num_games)):
    human_player = i % 2
    game = HexagonalPackit(4)
    # tree = MCTS(game, allow_transpositions = False, training = False)
    # tree.load("triangularPackit401.mcts")
    node = tree.root
    while not game.has_outcome():
        # game.render()
        actions = game.possible_actions()
        if game.current_player() == human_player:
            # print("Possible actions:")
            # for i, a in zip(actions, game.possible_actions_list):
            #     print(f'{i}: {a}')
            # action = int(input("> "))
            action = random.choice(actions)
            assert action in actions
            if node is not None:
                node = node.children[action] if action in node.children else None
        else:
            if node is not None and len(node.children) > 0:
                action = node.choose_best_action(tree.training)
                node = node.children[action]
            else:
                action = random.choice(actions)
                node = None
        game.take_action(action)
    # game.render()
    if 1-human_player in game.winner():
        wins[1-human_player] += 1
    
print(f'zaczynając: {wins[0] / (num_games/2)}, nie zaczynając: {wins[1] / (num_games/2)}')

100%|██████████| 100/100 [00:03<00:00, 29.56it/s]

zaczynając: 0.24, nie zaczynając: 0.86


In [6]:
import random
from copy import deepcopy
from tqdm import tqdm
from our_packit.mcts_games import HexagonalPackit
from mcts_simple import MCTS

wins = {
    0: 0,
    1: 0
}
game = HexagonalPackit(4)
tree = MCTS(game, allow_transpositions = False, training = False)
tree.load("hex42.mcts")
num_games = 100
for i in tqdm(range(num_games)):
    human_player = i % 2
    game = HexagonalPackit(4)
    # tree = MCTS(game, allow_transpositions = False, training = False)
    # tree.load("triangularPackit401.mcts")
    node = tree.root
    while not game.has_outcome():
        # game.render()
        actions = game.possible_actions()
        if game.current_player() == human_player:
            # print("Possible actions:")
            # for i, a in zip(actions, game.possible_actions_list):
            #     print(f'{i}: {a}')
            # action = int(input("> "))
            action = random.choice(actions)
            assert action in actions
            if node is not None:
                node = node.children[action] if action in node.children else None
        else:
            if node is not None and len(node.children) > 0:
                action = node.choose_best_action(tree.training)
                node = node.children[action]
            else:
                action = random.choice(actions)
                node = None
        game.take_action(action)
    # game.render()
    if 1-human_player in game.winner():
        wins[1-human_player] += 1
    
print(f'zaczynając: {wins[0] / (num_games/2)}, nie zaczynając: {wins[1] / (num_games/2)}')

100%|██████████| 100/100 [00:03<00:00, 29.75it/s]

zaczynając: 0.28, nie zaczynając: 0.76


In [7]:
import random
from copy import deepcopy
from tqdm import tqdm
from our_packit.mcts_games import HexagonalPackit
from mcts_simple import MCTS

wins = {
    0: 0,
    1: 0
}
game = HexagonalPackit(4)
tree = MCTS(game, allow_transpositions = False, training = False)
tree.load("hex41.mcts")
num_games = 100
for i in tqdm(range(num_games)):
    human_player = i % 2
    game = HexagonalPackit(4)
    # tree = MCTS(game, allow_transpositions = False, training = False)
    # tree.load("triangularPackit401.mcts")
    node = tree.root
    while not game.has_outcome():
        # game.render()
        actions = game.possible_actions()
        if game.current_player() == human_player:
            # print("Possible actions:")
            # for i, a in zip(actions, game.possible_actions_list):
            #     print(f'{i}: {a}')
            # action = int(input("> "))
            action = random.choice(actions)
            assert action in actions
            if node is not None:
                node = node.children[action] if action in node.children else None
        else:
            if node is not None and len(node.children) > 0:
                action = node.choose_best_action(tree.training)
                node = node.children[action]
            else:
                action = random.choice(actions)
                node = None
        game.take_action(action)
    # game.render()
    if 1-human_player in game.winner():
        wins[1-human_player] += 1
    
print(f'zaczynając: {wins[0] / (num_games/2)}, nie zaczynając: {wins[1] / (num_games/2)}')

100%|██████████| 100/100 [00:03<00:00, 29.40it/s]

zaczynając: 0.34, nie zaczynając: 0.88


In [5]:
import random
from copy import deepcopy
from tqdm import tqdm
from our_packit.mcts_games import HexagonalPackit
from mcts_simple import MCTS
for name in ['hex32.5v2.mcts', 'hex310v2.mcts', 'hex32v2.mcts', 'hex33v2.mcts']:
    wins = {
        0: 0,
        1: 0
    }
    game = HexagonalPackit(3)
    tree = MCTS(game, allow_transpositions = False, training = False)
    tree.load(name)
    num_games = 500
    for i in tqdm(range(num_games)):
        human_player = i % 2
        game = HexagonalPackit(3)
        # tree = MCTS(game, allow_transpositions = False, training = False)
        # tree.load("triangularPackit401.mcts")
        node = tree.root
        while not game.has_outcome():
            # game.render()
            actions = game.possible_actions()
            if game.current_player() == human_player:
                # print("Possible actions:")
                # for i, a in zip(actions, game.possible_actions_list):
                #     print(f'{i}: {a}')
                # action = int(input("> "))
                action = random.choice(actions)
                assert action in actions
                if node is not None:
                    node = node.children[action] if action in node.children else None
            else:
                if node is not None and len(node.children) > 0:
                    action = node.choose_best_action(tree.training)
                    node = node.children[action]
                else:
                    action = random.choice(actions)
                    node = None
            game.take_action(action)
        # game.render()
        if 1-human_player in game.winner():
            wins[1-human_player] += 1
    print(name)
    print(f'zaczynając: {wins[0] / (num_games/2)}, nie zaczynając: {wins[1] / (num_games/2)}')

100%|██████████| 500/500 [00:05<00:00, 92.43it/s]


hex32.5v2.mcts
zaczynając: 0.556, nie zaczynając: 0.832


100%|██████████| 500/500 [00:05<00:00, 95.61it/s] 


hex310v2.mcts
zaczynając: 0.648, nie zaczynając: 0.864


100%|██████████| 500/500 [00:05<00:00, 90.00it/s]


hex32v2.mcts
zaczynając: 0.496, nie zaczynając: 0.844


100%|██████████| 500/500 [00:05<00:00, 96.29it/s] 

hex33v2.mcts
zaczynając: 0.604, nie zaczynając: 0.9


In [7]:
import random
from copy import deepcopy
from tqdm import tqdm
from our_packit.mcts_games import HexagonalPackit
from mcts_simple import MCTS
for name in ['tri42.5v2.mcts', 'tri410v2.mcts', 'tri42v2.mcts', 'tri43v2.mcts']:
    wins = {
        0: 0,
        1: 0
    }
    game = TriangularPackit(4)
    tree = MCTS(game, allow_transpositions = False, training = False)
    tree.load(name)
    num_games = 500
    for i in tqdm(range(num_games)):
        human_player = i % 2
        game = TriangularPackit(4)
        # tree = MCTS(game, allow_transpositions = False, training = False)
        # tree.load("triangularPackit401.mcts")
        node = tree.root
        while not game.has_outcome():
            # game.render()
            actions = game.possible_actions()
            if game.current_player() == human_player:
                # print("Possible actions:")
                # for i, a in zip(actions, game.possible_actions_list):
                #     print(f'{i}: {a}')
                # action = int(input("> "))
                action = random.choice(actions)
                assert action in actions
                if node is not None:
                    node = node.children[action] if action in node.children else None
            else:
                if node is not None and len(node.children) > 0:
                    action = node.choose_best_action(tree.training)
                    node = node.children[action]
                else:
                    action = random.choice(actions)
                    node = None
            game.take_action(action)
        # game.render()
        if 1-human_player in game.winner():
            wins[1-human_player] += 1
    print(name)
    print(f'zaczynając: {wins[0] / (num_games/2)}, nie zaczynając: {wins[1] / (num_games/2)}')

100%|██████████| 500/500 [00:01<00:00, 270.29it/s]


tri42.5v2.mcts
zaczynając: 0.448, nie zaczynając: 0.968


100%|██████████| 500/500 [00:01<00:00, 277.21it/s]


tri410v2.mcts
zaczynając: 0.752, nie zaczynając: 0.992


100%|██████████| 500/500 [00:01<00:00, 273.27it/s]


tri42v2.mcts
zaczynając: 0.416, nie zaczynając: 0.964


100%|██████████| 500/500 [00:01<00:00, 268.01it/s]

tri43v2.mcts
zaczynając: 0.644, nie zaczynając: 0.992


In [35]:
['tri4', 'tri5', 'tri6', 'hex3', 'hex4'][0:4:3]

['tri4', 'hex3']

In [2]:
import random
from copy import deepcopy
from tqdm import tqdm
from our_packit.mcts_games import TriangularPackit
from mcts_simple import MCTS

wins = {
    0: 0,
    1: 0
}
game = TriangularPackit(4)
tree = MCTS(game, allow_transpositions = False, training = False)
tree.load("megamocnyTri4.mcts")
num_games = 500
for i in tqdm(range(num_games)):
    human_player = i % 2
    game = TriangularPackit(4)
    # tree = MCTS(game, allow_transpositions = False, training = False)
    # tree.load("triangularPackit401.mcts")
    node = tree.root
    while not game.has_outcome():
        # game.render()
        actions = game.possible_actions()
        if game.current_player() == human_player:
            # print("Possible actions:")
            # for i, a in zip(actions, game.possible_actions_list):
            #     print(f'{i}: {a}')
            # action = int(input("> "))
            action = random.choice(actions)
            assert action in actions
            if node is not None:
                node = node.children[action] if action in node.children else None
        else:
            if node is not None and len(node.children) > 0:
                action = node.choose_best_action(tree.training)
                node = node.children[action]
            else:
                action = random.choice(actions)
                node = None
        game.take_action(action)
    # game.render()
    if 1-human_player in game.winner():
        wins[1-human_player] += 1
    
print(f'zaczynając: {wins[0] / (num_games/2)}, nie zaczynając: {wins[1] / (num_games/2)}')

100%|██████████| 500/500 [00:01<00:00, 274.32it/s]

zaczynając: 0.66, nie zaczynając: 0.972


In [4]:
import random
from copy import deepcopy
from tqdm import tqdm
from our_packit.mcts_games import HexagonalPackit
from mcts_simple import MCTS

wins = {
    0: 0,
    1: 0
}
game = HexagonalPackit(3)
tree = MCTS(game, allow_transpositions = False, training = False)
tree.load("megamocnyHex3.mcts")
num_games = 500
for i in tqdm(range(num_games)):
    human_player = i % 2
    game = HexagonalPackit(3)
    # tree = MCTS(game, allow_transpositions = False, training = False)
    # tree.load("triangularPackit401.mcts")
    node = tree.root
    while not game.has_outcome():
        # game.render()
        actions = game.possible_actions()
        if game.current_player() == human_player:
            # print("Possible actions:")
            # for i, a in zip(actions, game.possible_actions_list):
            #     print(f'{i}: {a}')
            # action = int(input("> "))
            action = random.choice(actions)
            assert action in actions
            if node is not None:
                node = node.children[action] if action in node.children else None
        else:
            if node is not None and len(node.children) > 0:
                action = node.choose_best_action(tree.training)
                node = node.children[action]
            else:
                action = random.choice(actions)
                node = None
        game.take_action(action)
    # game.render()
    if 1-human_player in game.winner():
        wins[1-human_player] += 1
    
print(f'zaczynając: {wins[0] / (num_games/2)}, nie zaczynając: {wins[1] / (num_games/2)}')

100%|██████████| 500/500 [00:04<00:00, 100.97it/s]

zaczynając: 0.568, nie zaczynając: 0.92


In [ ]:
import random
from copy import deepcopy
from tqdm import tqdm

wins = {
    0: 0,
    1: 0
}
game = TriangularPackit(4)
tree = MCTS(game, allow_transpositions = False, training = False)
tree.load("triangularPackit401.mcts")
tree2 = MCTS(game, allow_transpositions = False, training = False)
tree2.load("triangularPackit402.mcts")
num_games = 1000
for i in tqdm(range(num_games)):
    human_player = i % 2
    game = TriangularPackit()
    # tree = MCTS(game, allow_transpositions = False, training = False)
    # tree.load("triangularPackit401.mcts")
    game = TriangularPackit()
    node = tree.root
    while not game.has_outcome():
        # game.render()
        actions = game.possible_actions()
        if game.current_player() == human_player:
            # print("Possible actions:")
            # for i, a in zip(actions, game.possible_actions_list):
            #     print(f'{i}: {a}')
            # action = int(input("> "))
            action = random.choice(actions)
            assert action in actions
            if node is not None:
                node = node.children[action] if action in node.children else None
        else:
            if node is not None and len(node.children) > 0:
                action = node.choose_best_action(tree.training)
                node = node.children[action]
            else:
                action = random.choice(actions)
                node = None
        game.take_action(action)
    # game.render()
    if 1-human_player in game.winner():
        wins[1-human_player] += 1
    
print(f'zaczynając: {wins[0] / (num_games/2)}, nie zaczynając: {wins[1] / (num_games/2)}')

In [31]:
import random
from copy import deepcopy

human_player = 0 # X

game = TriangularPackit()
tree = MCTS(game, allow_transpositions = False, training = False)
tree.load("triangularPackit.mcts")
game = TriangularPackit()
node = tree.root
while not game.has_outcome():
    game.render()
    actions = game.possible_actions()
    if game.current_player() == human_player:
        print("Possible actions:")
        for i, a in zip(actions, game.possible_actions_list):
            print(f'{i}: {a}')
        action = int(input("> "))
        assert action in actions
        if node is not None:
            node = node.children[action] if action in node.children else None
    else:
        if node is not None and len(node.children) > 0:
            action = node.choose_best_action(tree.training)
            node = node.children[action]
        else:
            action = random.choice(actions)
            node = None
    game.take_action(action)
game.render()

          /0\
        /0\0/0\
      /0\0/0\0/0\
    /0\0/0\0/0\0/0\
  /0\0/0\0/0\0/0\0/0\
Possible actions:
0: [[1 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0]]
1: [[0 0 0 0 0 0 0 0 0]
 [1 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0]]
2: [[0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0]
 [1 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0]]
3: [[0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0]
 [1 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0]]
4: [[0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0]
 [1 0 0 0 0 0 0 0 0]]
5: [[0 0 0 0 0 0 0 0 0]
 [0 0 1 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0]]
6: [[0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0]
 [0 0 1 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0]]
7: [[0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0]
 [0 0 1 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0]]
8: [[0 0 0 0 0 0 0 0

In [ ]:
import concurrent.futures
from our_packit.mcts_games import HexagonalPackit, TriangularPackit
from mcts_simple import UCT, MCTS

input_data = [TriangularPackit(4), TriangularPackit(5), TriangularPackit(6), HexagonalPackit(3), HexagonalPackit(4)] 
trees = [UCT(game, False, True) for game in input_data]
names = ['tri4', 'tri5', 'tri6', 'hex3', 'hex4']

def train_tree(tree, n, name):
    tree.self_play(n)
    name += '.mcts'
    tree.save(name)
    
n = 1
with concurrent.futures.ProcessPoolExecutor() as executor:
    futures = [executor.submit(train_tree, tree, n, name) for tree,name in zip(trees, names)]
    results = [future.result() for future in concurrent.futures.as_completed(futures)]